In [43]:
import pennylane as qml
from pprop import Propagator
import numpy as np
from jax import jit, grad, value_and_grad
import optax  # optional but great for gradient-based optimization
import jax.numpy as jnp

In [2]:
side : int = 5
J : float = 1
h : float = 1

num_qubits : int = side * side

In [3]:
def hamiltonian(side: int, J: float, h: float) -> qml.Hamiltonian:
    coeffs = []
    obs = []

    # Nearest-neighbor ZZ interactions
    for x in range(side):
        for y in range(side):
            i = x * side + y

            # Right neighbor
            if y < side - 1:
                j = x * side + (y + 1)
                coeffs.append(-J / num_qubits)
                obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

            # Down neighbor
            if x < side - 1:
                j = (x + 1) * side + y
                coeffs.append(-J / num_qubits)
                obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # Transverse-field X terms
    for i in range(num_qubits):
        coeffs.append(-h / num_qubits)
        obs.append(qml.PauliX(i))

    return qml.Hamiltonian(coeffs, obs)

$$
H(J, h) = - \frac{1}{N}\left(
    J\sum_{\langle i,j \rangle} Z(i)Z(j) - h\sum_i X(i)
\right) 
$$

<img src="./assets/ising2d.svg" width="1000">

In [4]:
def circuit(params):
    index = 0

    # Initial RY and RX
    for q in range(num_qubits):
        qml.RY(params[index], wires=q)
        index += 1
        qml.RX(params[index], wires=q)
        index += 1

    # Horizontal entanglers
    for d in range(2):
        y_start = 0 if d % 2 == 0 else 1
        for x in range(side):
            for y in range(y_start, side - 1, 2):
                i = x * side + y
                j = x * side + (y + 1)
                qml.CNOT(wires=[i, j])

    # RX layer
    for q in range(num_qubits):
        qml.RX(params[index], wires=q)
        index += 1

    # Vertical entanglers
    for d in range(2):
        x_start = 0 if d % 2 == 0 else 1
        for y in range(side):
            for x in range(x_start, side - 1, 2):
                i = x * side + y
                j = (x + 1) * side + y
                qml.CNOT(wires=[i, j])

    # RX layer
    for q in range(num_qubits):
        qml.RX(params[index], wires=q)
        index += 1

    # Final RY
    for q in range(num_qubits):
        qml.RY(params[index], wires=q)
        index += 1
        
    return qml.expval(hamiltonian(side, J, h))

In [5]:
prop = Propagator(circuit, k1=6, k2=20)

In [6]:
prop

Propagator
  Number of qubits : 25
  Trainable parameters : 125

In [7]:
prop.show()

 0: ──RY(0.00)───RX(1.00)──╭●──RX(50.00)────────────╭●──RX(75.00)──RY(100.00)──────────── ···
 1: ──RY(2.00)───RX(3.00)──╰X─╭●──────────RX(51.00)─│──╭●──────────RX(76.00)───RY(101.00) ···
 2: ──RY(4.00)───RX(5.00)──╭●─╰X──────────RX(52.00)─│──│──────────╭●───────────RX(77.00)─ ···
 3: ──RY(6.00)───RX(7.00)──╰X─╭●──────────RX(53.00)─│──│──────────│───────────╭●───────── ···
 4: ──RY(8.00)───RX(9.00)─────╰X──────────RX(54.00)─│──│──────────│───────────│────────── ···
 5: ──RY(10.00)──RX(11.00)─╭●──RX(55.00)────────────╰X─│──────────│───────────│────────── ···
 6: ──RY(12.00)──RX(13.00)─╰X─╭●──────────RX(56.00)────╰X─────────│───────────│────────── ···
 7: ──RY(14.00)──RX(15.00)─╭●─╰X──────────RX(57.00)───────────────╰X──────────│────────── ···
 8: ──RY(16.00)──RX(17.00)─╰X─╭●──────────RX(58.00)───────────────────────────╰X───────── ···
 9: ──RY(18.00)──RX(19.00)────╰X──────────RX(59.00)────────────────────────────────────── ···
10: ──RY(20.00)──RX(21.00)─╭●──RX(60.00)────────────╭●──────

In [8]:
prop.propagate()

Propagating -0.04*Z0 Z1 + -0.04*Z0 Z5 + -0.04*Z1 Z2 + -0.04*Z1 Z6 + -0.04*Z2 Z3 + -0.04*Z2 Z7 + -0.04*Z3 Z4 + -0.04*Z3 Z8 + -0.04*Z4 Z9 + -0.04*Z5 Z6 + -0.04*Z5 Z10 + -0.04*Z6 Z7 + -0.04*Z6 Z11 + -0.04*Z7 Z8 + -0.04*Z7 Z12 + -0.04*Z8 Z9 + -0.04*Z8 Z13 + -0.04*Z9 Z14 + -0.04*Z10 Z11 + -0.04*Z10 Z15 + -0.04*Z11 Z12 + -0.04*Z11 Z16 + -0.04*Z12 Z13 + -0.04*Z12 Z17 + -0.04*Z13 Z14 + -0.04*Z13 Z18 + -0.04*Z14 Z19 + -0.04*Z15 Z16 + -0.04*Z15 Z20 + -0.04*Z16 Z17 + -0.04*Z16 Z21 + -0.04*Z17 Z18 + -0.04*Z17 Z22 + -0.04*Z18 Z19 + -0.04*Z18 Z23 + -0.04*Z19 Z24 + -0.04*Z20 Z21 + -0.04*Z21 Z22 + -0.04*Z22 Z23 + -0.04*Z23 Z24 + -0.04*X0 + -0.04*X1 + -0.04*X2 + -0.04*X3 + -0.04*X4 + -0.04*X5 + -0.04*X6 + -0.04*X7 + -0.04*X8 + -0.04*X9 + -0.04*X10 + -0.04*X11 + -0.04*X12 + -0.04*X13 + -0.04*X14 + -0.04*X15 + -0.04*X16 + -0.04*X17 + -0.04*X18 + -0.04*X19 + -0.04*X20 + -0.04*X21 + -0.04*X22 + -0.04*X23 + -0.04*X24


In [40]:
# Once propagated, we can estimate the output wavefunction energy of the circuit with the given input parameters
prop(np.random.rand(prop.num_params))

Array([-0.29190403], dtype=float32)

In [ ]:
# Todo: Training loop